In [61]:
from pydantic import BaseModel
from typing import Annotated, List, TypedDict
from langgraph.graph import StateGraph, START,END
from langchain_groq import ChatGroq
from langgraph.graph.message import add_messages
from langchain_core.messages import AnyMessage

from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition

from langgraph.checkpoint.memory import MemorySaver

from IPython.display import Image, display

In [62]:
model = ChatGroq(model="llama-3.1-8b-instant",temperature=0)

In [64]:
from langchain_core.messages import BaseMessage, ToolMessage
from langchain_groq import ChatGroq
from langgraph.graph import StateGraph, START, END
from pydantic import BaseModel

class StateSchema(BaseModel):
    questions:str
    answer:str
    notes:str
    
class InPutStateSchema(BaseModel):
    questions:str

class OutPutStateSchema(BaseModel):
    answer:str

model = ChatGroq(model="llama-3.1-8b-instant",temperature=0)

def input_tool_func(state:InPutStateSchema) -> StateSchema:
    print(state.model_dump_json(indent=2))
    return StateSchema(
        questions=state.questions,
        answer="sample_ans",
        notes = "sample_notes"
    )
    
def output_tool_func(state:StateSchema) -> OutPutStateSchema:
    print(state.model_dump_json(indent=2))
    return OutPutStateSchema(
        answer="this is the sample answer"
    )
    
tools = [input_tool_func,output_tool_func]
llm_with_tools = model.bind_tools(tools)

builder = StateGraph(StateSchema,input=InPutStateSchema,output=OutPutStateSchema)

builder.add_node("input_node",input_tool_func)
builder.add_node("output_node",output_tool_func)

builder.add_edge(START,"input_node")
builder.add_edge("input_node","output_node")
builder.add_edge("output_node",END)

graph = builder.compile()

# Invoke the graph with an instance of InPutStateSchema
result = graph.invoke(InPutStateSchema(questions="sample_questions"))
print(result)

{
  "questions": "sample_questions"
}
{
  "questions": "sample_questions",
  "answer": "sample_ans",
  "notes": "sample_notes"
}
{'answer': 'this is the sample answer'}
